In [74]:
import torch
from torch import nn
from torch.nn import functional as F

from torch.nn import Conv2d

In [75]:
enc_features = torch.randn(1, 64, 256, 256)

dec_features = torch.randn(1, 128, 256, 256)

In [76]:
conv2d = Conv2d(in_channels=64, out_channels=1, kernel_size=1)

In [77]:
conv2d_features = conv2d(enc_features)

In [78]:
conv2d_features.shape

torch.Size([1, 1, 256, 256])

In [79]:
class ChannelToSingleMax(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, enc_features):
        """
        Input shape: (N, C, H, W)
        Output shape: (N, 1, H, W)
        """
        return torch.max(enc_features, dim=1, keepdim=True)[0]

In [121]:
torch.max(enc_features, dim=1, keepdim=True).values.shape

torch.Size([1, 1, 256, 256])

In [80]:
pool = ChannelToSingleMax()
mx_features = pool(enc_features)
mx_features.shape

torch.Size([1, 1, 256, 256])

In [81]:
class ChannelAveragePool(nn.Module):
    def __init__(self):
        super(ChannelAveragePool, self).__init__()

    def forward(self, enc_features):
        return torch.mean(enc_features, dim=1, keepdim=True)


In [82]:
avg_pooling = ChannelAveragePool()
avg_features = avg_pooling(enc_features)
avg_features.shape

torch.Size([1, 1, 256, 256])

In [83]:
conv2d_features.shape, mx_features.shape, avg_features.shape

(torch.Size([1, 1, 256, 256]),
 torch.Size([1, 1, 256, 256]),
 torch.Size([1, 1, 256, 256]))

In [113]:
concat_enc_features = torch.cat([conv2d_features, mx_features, avg_features], dim=1)
concat_enc_features.shape

torch.Size([1, 3, 256, 256])

In [85]:
con_7x7 = Conv2d(in_channels=3, out_channels=1, kernel_size=7, padding=3)
output = con_7x7(concat_enc_features)
output.shape

torch.Size([1, 1, 256, 256])

In [87]:
dec_features = torch.randn(1, 128, 256, 256)

In [92]:
dec_conv2d = Conv2d(in_channels=128, out_channels=1, kernel_size=1)
dec_conv2d_features = dec_conv2d(dec_features)
dec_conv2d_features.shape

torch.Size([1, 1, 256, 256])

In [93]:
# class ChannelToSingleMax(nn.Module):
#     def __init__(self):
#         super().__init__()
        
#     def forward(self, enc_features):
#         """
#         Input shape: (N, C, H, W)
#         Output shape: (N, 1, H, W)
#         """
#         return torch.max(enc_features, dim=1, keepdim=True)[0]

In [114]:
pooling = ChannelToSingleMax()
mx_features = pooling(dec_conv2d_features)
mx_features.shape

torch.Size([1, 1, 256, 256])

In [95]:
# class ChannelAveragePool(nn.Module):
#     def __init__(self):
#         super(ChannelAveragePool, self).__init__()

#     def forward(self, enc_features):
#         return torch.mean(enc_features, dim=1, keepdim=True)

In [115]:
pooling2 = ChannelAveragePool()
avg_features = pooling2(dec_conv2d_features)
dec_conv2d_features.shape

torch.Size([1, 1, 256, 256])

In [116]:
dec_conv2d_features.shape, mx_features.shape, avg_features.shape

(torch.Size([1, 1, 256, 256]),
 torch.Size([1, 1, 256, 256]),
 torch.Size([1, 1, 256, 256]))

In [117]:
concat_dec_features = torch.cat([dec_conv2d_features, mx_features, avg_features], dim=1)
concat_dec_features.shape

torch.Size([1, 3, 256, 256])

In [118]:
con2d7x7 = Conv2d(in_channels=3, out_channels=1, kernel_size=7, padding=3)
output1 = con2d7x7(concat_dec_features)
output1.shape

torch.Size([1, 1, 256, 256])

In [119]:
output2 = output + output1
output2.shape

torch.Size([1, 1, 256, 256])

In [120]:
sigmoid = nn.Sigmoid()
output3 = sigmoid(output2)
output3.shape

torch.Size([1, 1, 256, 256])